## Testes com IEEE 34 Bus
_Autor: Leonardo Jaime_

### Imports

In [91]:
# !pip install py-dss-interface

# instalação da biblioteca do OpenDSS

# as outras bibliotecas estão inclusas no anaconda

In [92]:
# Importação das bibliotecas utilizadas
import py_dss_interface
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from bokeh.io import show, curdoc
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file
from bokeh.models import Range1d
from bokeh.themes import built_in_themes

Repósitorio GitHub: <a href="https://github.com/leojms/OpenDSS">Repositório</a> 

### Funções

##### Variáveis

In [93]:
# usuario utilizado (esse método só funciona comigo mesmo kkkk)
# caso use em outro computador, apenas substituir o local e diretorio por endereços da sua maquina
u_senai = "leonardo.simoes"
u_note = "leona"
# Diretório do arquivo do alimentador
# cas-o seja utilizado um outro alimentador, ou em outra máquina, é necessário criar a variável com o novo diretório
diretorio = fr'C:\Users\{u_senai}\OneDrive - Sistema FIEB\centro_comp\opendss\alimentadores'
#############################
# IMPORTANTE

# O endereço local 2 refere-se a pasta anterior, onde serão exportados o gráfico de perdas em html e o arquivo excel
# com os dados dos cenários
# O local se refere a onde fica o arquivo do circuito e a pasta com os arquivos das curvas
# Salvar a pasta com os arquivos do circuito que fica na pasta do opendss em outro lugar, de maneira a evitar erros de adm
#############################

In [94]:
# variaveis globais
dss = py_dss_interface.DSSDLL()
periodo = 24
horario = 0
linha_13 = "line.650632"
linha_34 ="line.L1"
bus_13 = 680
bus_34 = 840
cenarios_13 = [0, 75, 500, 2000, 3000, 5000]
cenarios_34 = [0, 75, 500, 1200, 2200, 3500]
kv_13 = 4.16
kv_34 = 24.9
local_13 = fr"{diretorio}\IEEE\13Bus"
local_34 = fr"{diretorio}\IEEE\34Bus"
nome_arquivo_34 = "Run_IEEE34Mod1.dss"
arquivo_34 = fr"{local_34}\{nome_arquivo_34}"
nome_arquivo_13 = "IEEE13Nodeckt.dss"
arquivo_13 = fr"{local_13}\{nome_arquivo_13}"
# caso seja executado em outra máquina,
# baixar os arquivos das curvas via github e criar variáveis para o novo endereço do arquivo

OpenDSS Started successfully! 
OpenDSS Version 9.4.0.1 (64-bit build); License Status: Open 




##### Elementos

In [95]:
# Criação dos elementos

def medidor(linha):
    # criando o objeto medidor via opendss, na linha de saida do transformador
    dss.text(f"new Energymeter.medidor {linha} terminal=1")
    # adicionando um marcador em uma barra escolhida, apenas para demostrar como faz
    # dss.text("AddBusMarker bus=680 color=red size=6 code=15")


def carga(local):
    # criando a curva de carga, baseada na residencial
    # e editando todas as cargas, para que elas tenham esse padrao de comportamento
    dss.text("new loadshape.dia npts=24 interval=1")
    dss.text(fr"~ mult=(file=[{local}\curvas\curva_residencial.txt])")
    dss.text("batchedit load..* daily=dia")
    # curva alternativa
    # dss.text(r"~ mult=(file=[C:\Users\leonardo.simoes\OneDrive - Sistema FIEB\centro_comp\opendss\alimentadores\IEEE\13Bus\curvas\curva_res_red.txt])")


def fv1(kw, bus, kv, local):
    if kw == 0:
        return
    else:
        # criação das curvas do sistema fotovoltaico
        # curva do comportamento do fator de temperatura de acordo com o aumento da temperatura
        dss.text("new XYcurve.ctemp npts=4 xarray=[0 25 75 100] yarray=[1.2 1.0 0.8 0.6]")
        # Curva de eficiência do inversor
        dss.text("new XYcurve.ceficiencia npts=5 xarray=[0.1 0.2 0.4 1 2] yarray=[0.86 0.9 0.93 0.97 0.8]")
        # Curva de irradiancia diaria baseada na irradiancia solar local
        dss.text("new loadshape.cirrad npts=24 interval=1")
        dss.text(fr"~ mult=(file=[{local}\curvas\curva_fv.txt])")
        # curva da variação de temperatura da placa fotovoltaica diaria
        dss.text("new tshape.t npts=24 interval=1")
        dss.text(fr"~ temp=(file=[{local}\curvas\curva_temp.txt])")
        # criação do sistema fotovoltaico
        # criação do objeto, associação das curvas criadas e criação dos monitores
        dss.text(f"new PVSystem.fv phases=3 bus1={bus} kv={kv} irrad=0.98 pmpp={kw} kva=6000 temperature=25 pf=1")
        dss.text("~ %cutin=0.1 %cutout=0.1 effcurve=ceficiencia P-tCurve=ctemp Daily=cirrad Tdaily=t")
        dss.text(f"new monitor.fv_power element=PVSystem.fv terminal=1 mode=1 ppolar=no")
        dss.text(f"new monitor.fv_variaveis element=PVSystem.fv terminal=1 mode=3")

##### Cenários

In [96]:
def cenario(arquivo, horas, kw, bus, kv, linha, local):
    # cenario 3, utilização do sistema fotovoltaico associado ao v2g
    # função resolve o fator de potência e exporta os dados de perdas e potência total diária
    # limpar o buffer
    dss.text("clear")
    # compilar o arquivo que contém o alimentador
    dss.text(f"compile [{arquivo}]")
    # chamar as funções com medidor, carga e os elementos de gd se tiverem
    medidor(linha)
    carga(local)
    fv1(kw, bus, kv, local)
    # configuração de simulação e resolução do fator de potência
    dss.text("set mode = daily")
    dss.text("set stepsize = 1h")
    dss.text("set number = 1")
    dss.text(f"set hour={horas - 1}")
    dss.solution_solve()
    # extração dos valores de perdas, potência e calculo das perdas percentuais
    dss.circuit_set_active_element(linha)
    perdas = dss.circuit_losses()[0] / 10**3
    pot = dss.cktelement_powers()[0]
    return perdas, pot

##### Exports

In [97]:
def excel(dataframe, nome_arquivo, sep):
    # função para exportar os dados dos cenarios para excel e gerar os dados para o dashboard
    nome = f'{nome_arquivo}.xlsx'
    dataframe.to_excel(fr'{diretorio}\cen_{sep}_bus\{nome}')
    print(f"O arquivo de dados {nome} foi salvo em excel com sucesso")


def monitores_dss(arquivo, sep, cenario_flag, cenarios, bus, kv, linha, local):
    # função para exportar os monitores do/dos sistema(s) fotovoltaico(s) de cada cenario
    # compilar arquivo
    dss.text(f"compile [{arquivo}]")
    # escolher cenario
    cenario(arquivo, horario, kw = cenarios[cenario_flag], bus=bus, kv=kv, linha=linha, local=local)
    # rodar simulação no regime determinado
    dss.text("set mode = daily")
    dss.text("set stepsize = 1h")
    dss.text(f"set number = {periodo}")
    # resolver o fator de potência
    dss.solution_solve()
    # condicional para plotar os monitores, de acordo com os monitores que forem desejados
    if cenario_flag == 0:
        print("Nao ha monitores a serem mostrados")
    elif cenario_flag>0 and cenario_flag<=5:
        dss.monitors_write_name("fv_variaveis")
        var1 = dss.monitors_channel(1)
        var2 = dss.monitors_channel(2)
        var3 = dss.monitors_channel(3)
        var4 = dss.monitors_channel(4)
        output_file(fr'{diretorio}\fv_{sep}_bus.html')
        curdoc().theme = 'dark_minimal'
        irrad = figure(x_axis_label="Horas (h)", title="Irradiância (pu/m²)")
        irrad.x_range = Range1d(0, 24)
        irrad.line(x=range(1, len(var1)+1), y=var1, color='lime')
        pot = figure(x_axis_label="Horas (h)", title="Potência (kW)")
        pot.x_range = Range1d(0, 24)
        pot.line(x=range(1, len(var2)+1), y=var2, color='cyan')
        temp = figure(x_axis_label="Horas (h)", title="Fator de temperatura")
        temp.x_range = Range1d(0, 24)
        temp.line(x=range(1, len(var3)+1), y=var3, color='yellow')
        eff = figure(x_axis_label="Horas (h)", title="Eficiência")
        eff.x_range = Range1d(0, 24)
        eff.line(x=range(1, len(var4)+1), y=var4, color='hotpink')
        grid_layout = gridplot([[irrad, pot], [temp, eff]])
        show(grid_layout)
    else:
        print("O numero digitado para o cenario eh incorreto")

    
def plots_horario(arquivo, horario, cenario_flag, cenarios, bus, kv, linha, local):
    # função para plotar e exportar dados referentes a algum horario especifico de algum cenario especifico
    # localização do diretorio de onde o opendss exporta as sobrecargas
    # caso seja modificado, criar outra variável com o novo endereço, que sera o endereço de onde está o arquivo do alimentador.dss
    # diretorio_senai = fr"{local}\IEEE13Nodeckt_EXP_OVERLOADS.CSV"
    # compilar o arquivo que contém o alimentador
    dss.text(f"compile [{arquivo}]")
    # condicional para escolher o cenario
    cenario(arquivo, horario, kw = cenarios[cenario_flag], bus=bus, kv=kv, linha=linha, local=local)
    # Plotar o circuito alimentador
    dss.text("plot circuit Power max=2000 y y C1=$00FF0000")
    dss.text("Plot Profile Phases = All")
    # exportar em txt os elementos com sobrecarga
    # dss.text("Export overloads")
    # abrir o txt exportado acima, de acordo com o diretorio
    # dss.text(fr"fileedit [{diretorio_senai}]")
    

def plot_perdas(dados, label_base, label1, label2, label3, label4, label5, sep):
    # função para plotar os gráficos dos dados de perdas e potencia extraídos dos cenários
    # plots através da biblioteca bokeh, que apresenta gráficos interativos e exporta para html
    output_file(fr'{diretorio}\{sep}_bus.html')
    curdoc().theme = 'dark_minimal'
    perdas = figure(x_axis_label="Horas (h)", title="Perdas (kW)")
    perdas.x_range = Range1d(0, 24)
    perdas.line(x=dados["horas (h)"], y=dados["perdas em kW Cenario base"], color='white', legend_label=label_base)
    perdas.line(x=dados["horas (h)"], y=dados["perdas em kW Cenario 1"], color='hotpink', legend_label=label1)
    perdas.line(x=dados["horas (h)"], y=dados["perdas em kW Cenario 2"], color='deepskyblue', legend_label=label2)
    perdas.line(x=dados["horas (h)"], y=dados["perdas em kW Cenario 3"], color='yellow', legend_label=label3)
    perdas.line(x=dados["horas (h)"], y=dados["perdas em kW Cenario 4"], color='lawngreen', legend_label=label4)
    perdas.line(x=dados["horas (h)"], y=dados["perdas em kW Cenario 5"], color='aquamarine', legend_label=label5)
    potencia = figure(x_axis_label="Horas (h)", title="Potencia Total (kW)")
    potencia.x_range = Range1d(0, 24)
    potencia.line(x=dados["horas (h)"], y=dados["potencia em kW Cenario base"], color='white', legend_label=label_base)
    potencia.line(x=dados["horas (h)"], y=dados["potencia em kW Cenario 1"], color='hotpink', legend_label=label1)
    potencia.line(x=dados["horas (h)"], y=dados["potencia em kW Cenario 2"], color='deepskyblue', legend_label=label2)
    potencia.line(x=dados["horas (h)"], y=dados["potencia em kW Cenario 3"], color='yellow', legend_label=label3)
    potencia.line(x=dados["horas (h)"], y=dados["potencia em kW Cenario 4"], color='lawngreen', legend_label=label4)
    potencia.line(x=dados["horas (h)"], y=dados["potencia em kW Cenario 5"], color='aquamarine', legend_label=label5)
    grid_layout = gridplot([[perdas, potencia]])
    show(grid_layout)
    

def dados_fv(arquivo, sep, cenario_flag, cenarios, bus, kv, linha, local):
    dss.text(f"compile [{arquivo}]")
    dia = range(0, 24)
    d = dict()

    for cenario_flag in range(1, 6):
        cenario(arquivo, horario, kw = cenarios[cenario_flag], bus=bus, kv=kv, linha=linha, local=local)
        dss.text("set mode = daily")
        dss.text("set stepsize = 1h")
        dss.text(f"set number = {periodo}")
        dss.solution_solve()
        dss.monitors_write_name("fv_variaveis")
        var1 = dss.monitors_channel(1)
        var2 = dss.monitors_channel(2)
        var3 = dss.monitors_channel(3)
        var4 = dss.monitors_channel(4)
        d[f"Irradiância Cenário {cenario} (pu/m²)"] = var1
        d[f"Potência Cenário {cenario} (kW)"] = var2
        d[f"Fator de temperatura Cenário {cenario}"] = var3
        d[f"Eficiência Cenário {cenario}"] = var4
        
    d[f"Irradiância Cenário Base (pu/m²)"] = np.zeros(24)
    d[f"Potência Cenário Base (kW)"] = np.zeros(24)
    d[f"Fator de temperatura Cenário Base"] = np.zeros(24)
    d[f"Eficiência Cenário Base"] = np.zeros(24)     
    df = pd.DataFrame().from_dict(d)
    nome = f'Variáveis FV.xlsx'
    df.to_excel(fr'{diretorio}\cen_{sep}_bus\{nome}')
    print(f"O arquivo de dados {nome} foi salvo em excel com sucesso")

##### Processo

In [98]:
def listagem(lista_geral, perdas, potencia):
    # Organização dos dados de cada cenário em listas para cada variável
    perdas.append(lista_geral[0])
    potencia.append(lista_geral[1])


def organiza(arquivo, sep, cenarios, bus, kv, linha, local):
    # função que chama as funções de processo criadas acima
    # criação do dia com as 24 horas
    dia = range(0, 24)
    dicio = dict()
    dicio["horas (h)"] = dia
    
    n_cenarios = len(cenarios)
    
    # Organização dos dados nas listas referentes a cada variável
    for cenario_flag in range(0, n_cenarios):
        # criação das listas
        lista_perdas = list()
        lista_pot = list()
        for hora in dia:
            listagem(cenario(arquivo, hora, kw=cenarios[cenario_flag], bus=bus, kv=kv, linha=linha, local=local), lista_perdas, lista_pot)
        if cenario_flag == 0:
            dicio["perdas em kW Cenario base"] = lista_perdas
            dicio["potencia em kW Cenario base"] = lista_pot
        else:
            dicio[f"perdas em kW Cenario {cenario_flag}"] = lista_perdas
            dicio[f"potencia em kW Cenario {cenario_flag}"] = lista_pot
    
    
    dados = pd.DataFrame().from_dict(dicio)

    # Plot dos cenários de perdas e potência
    plot_perdas(dados, "Cenario Base", "Cenario 1", "Cenario 2", "Cenario 3", "Cenario 4", "Cenario 5", sep)

    # exportação do banco de dados para excel
    excel(dados, "todos os cenarios", sep)

def processo(sep, hor, cen):
    if sep == 13:
        arquivo = arquivo_13
        bus = bus_13
        kv = kv_13
        linha = linha_13
        cenarios = cenarios_13
        local = local_13
    elif sep == 34:
        arquivo = arquivo_34
        bus = bus_34
        kv = kv_34
        linha = linha_34
        cenarios = cenarios_34
        local = local_34
    else:
        pass
    # Função para chamar a função acima e para chamar as outras funções de plot criadas
    organiza(arquivo, sep=sep, cenarios=cenarios, bus=bus, kv=kv, linha=linha, local=local)
    plots_horario(arquivo, horario=hor, cenario_flag=cen, cenarios=cenarios, bus=bus, kv=kv, linha=linha, local=local)
    monitores_dss(arquivo, sep=sep, cenario_flag=cen, cenarios=cenarios, bus=bus, kv=kv, linha=linha, local=local)
    dados_fv(arquivo, sep=sep, cenario_flag=cen, cenarios=cenarios, bus=bus, kv=kv, linha=linha, local=local)

### Calcular Fluxo

In [99]:
# função para executar os comandos de todas as funções criadas no código
processo(sep=34, cen=3, hor=12)
processo(sep=13, cen=3, hor=12)

O arquivo de dados todos os cenarios.xlsx foi salvo em excel com sucesso
O arquivo de dados Variáveis FV.xlsx foi salvo em excel com sucesso
O arquivo de dados todos os cenarios.xlsx foi salvo em excel com sucesso
O arquivo de dados Variáveis FV.xlsx foi salvo em excel com sucesso
